In [9]:
import os
import pandas as pd

In [10]:
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

In [11]:
files = list(set(os.listdir('backend/data/odds')).difference({'odds.csv', 'odds.ipynb', 'nfl odds 2022-23.xlsx', 'nfl odds 2021-22.xlsx'}))
files

['nfl odds 2012-13.xlsx',
 'nfl odds 2011-12.xlsx',
 'nfl odds 2010-11.xlsx',
 'nfl odds 2018-19.xlsx',
 'nfl odds 2019-20.xlsx',
 'nfl odds 2015-16.xlsx',
 'nfl odds 2013-14.xlsx',
 'nfl odds 2020-21.xlsx',
 'nfl odds 2017-18.xlsx',
 'nfl odds 2014-15.xlsx',
 'nfl odds 2016-17.xlsx']

In [12]:
def spreads(col1, col2):
    if col1 == "pk":
        col1 = 0
    if col2 == "pk":
        col2 = 0
    if col1 < col2:
        return col1
    else:
        return col2

In [13]:
def totals(col1, col2):
    if col1 == "pk":
        col1 = 0
    if col2 == "pk":
        col2 = 0
    if col1 < col2:
        return col2
    else:
        return col1

In [14]:
def odds_season(filename):
    df = pd.read_excel(f'backend/data/odds/{filename}')
    df.columns = [col.lower() for col in df.columns]

    df["year"] = filename.split("-")[0][-4:]

    home_df = df[df["vh"] == "H"].reset_index()[["team", "ml", "close", "year", "week"]]
    home_df.rename(columns={"team": "home"}, inplace=True)

    away_df = df[df["vh"] == "V"].reset_index()[["team", "ml", "close", "year", "week"]]
    away_df.rename(columns={"team": "away"}, inplace=True)

    df = home_df.join(away_df, lsuffix="_h", rsuffix="_a")

    df['spread'] = df.apply(lambda x: spreads(x.close_h, x.close_a), axis=1)
    df['total'] = df.apply(lambda x: totals(x.close_h, x.close_a), axis=1)
    df.drop(["close_h", "close_a"], axis=1, inplace=True)

    df.drop(["week_a", "year_a"], axis=1, inplace=True)
    df.rename(columns={"year_h": "year", "week_h": "week"}, inplace=True)

    teams_dict = {
        'Philadelphia': 'philadelphia-eagles', 'St.Louis': 'st-louis-rams',
        'TampaBay': 'tampa-bay-buccaneers', 'NYGiants': 'new-york-giants', 'GreenBay': 'green-bay-packers',
        'Chicago': 'chicago-bears', 'NewEngland': 'new-england-patriots', 'Pittsburgh': 'pittsburgh-steelers',
        'Houston': 'houston-texans', 'Denver': 'denver-broncos', 'SanFrancisco': 'san-francisco-49ers',
        'Minnesota': 'minnesota-vikings', 'Washington': 'washington-redskins',
        'Jacksonville': 'jacksonville-jaguars', 'Tennessee': 'tennessee-titans',
        'Carolina': 'carolina-panthers', 'KansasCity': 'kansas-city-chiefs', 'Miami': 'miami-dolphins',
        'Atlanta': 'atlanta-falcons', 'NewOrleans': 'new-orleans-saints', 'Baltimore': 'baltimore-ravens',
        'Seattle': 'seattle-seahawks', 'SanDiego': 'san-diego-chargers', 'Dallas': 'dallas-cowboys',
        'Cincinnati': 'cincinnati-bengals', 'NYJets': 'new-york-jets', 'Detroit': 'detroit-lions',
        'Arizona': 'arizona-cardinals', 'Oakland': 'oakland-raiders', 'Indianapolis': 'indianapolis-colts',
        'Buffalo': 'buffalo-bills', 'Cleveland': 'cleveland-browns', 'LARams': 'los-angeles-rams',
        'LAChargers': 'los-angeles-chargers', 'washington': 'washington-football-team',
        'LasVegas': 'las-vegas-raiders'
    }

    df["home"] = df["home"].apply(lambda x: teams_dict[x])
    df["away"] = df["away"].apply(lambda x: teams_dict[x])

    return df

In [15]:
odds = pd.DataFrame()
for file in files:
    odds = odds.append(odds_season(file))
odds = pd.concat(
    [
        odds, 
        pd.read_excel('backend\\data\\odds\\nfl odds 2021-22.xlsx'),
        pd.read_excel('backend\\data\\odds\\nfl odds 2022-23.xlsx')
    ],
    axis=0
)
odds["home"] = odds["home"].apply(lambda x: ' '.join([word.capitalize() for word in x.split('-')]))
odds["away"] = odds["away"].apply(lambda x: ' '.join([word.capitalize() for word in x.split('-')]))
odds.rename({'year': 'season'}, axis=1, inplace=True)

odds.tail(20)

,home,ml_h,season,week,away,ml_a,spread,total
281,Kansas City Chiefs,-130,2021,20,Buffalo Bills,110,2.5,54.0
282,Kansas City Chiefs,-350,2021,21,Cincinnati Bengals,270,7.0,54.5
283,Los Angeles Rams,-180,2021,21,San Francisco 49ers,155,3.5,45.5
284,Cincinnati Bengals,160,2021,22,Los Angeles Rams,-190,4.0,48.5
0,Los Angeles Rams,120,2022,1,Buffalo Bills,-134,2.5,51.5
1,Atlanta Falcons,190,2022,1,New Orleans Saints,-205,5.0,41.0
2,Chicago Bears,235,2022,1,San Francisco 49ers,-275,6.5,42.0
3,Cincinnati Bengals,-270,2022,1,Pittsburgh Steelers,245,6.5,44.5
4,Detroit Lions,180,2022,1,Philadelphia Eagles,-180,3.5,46.5
5,Miami Dolphins,-140,2022,1,New England Patriots,123,3.0,44.5


In [16]:
odds.to_csv('backend\\data\\odds\\odds.csv', index=False)